In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mne
import os
import sys
from mne.datasets import eegbci
import glob
from IPython.display import clear_output
import numpy as np
import torch
from torch import nn
import torch.optim as optim

from mne.datasets import eegbci
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader,SubsetRandomSampler
from scipy import signal

import torch
import torch.cuda as cuda
import torch.nn as nn
from torch.autograd import Variable
import math
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

from utils import create_dataloader
from dataset import EEG
import wandb
from mne.datasets import sample
from sklearn import preprocessing
import os
import sys


sys.path.append(os.path.abspath(os.path.join('..')))
# Now do your import
from config import *

c:\Users\Pongk\.virtualenvs\EEG-python-MGW7v3UV\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from model import ConvNet,CNN2D,ConvNet_physionet,ConvNet2
from torchsummary import summary
net = ConvNet2().cuda()


In [3]:
# execute
#net.load_state_dict(torch.load('./save_weight/sunsun/sunsun_85.1852'))
online_path = "/Users/Pongk/Desktop/Work/mi-project/EEG-python/Training/save_weight/S18_newConv_irr_-2-5_8-13_3ch/0.6852_S18_newConv_irr_-2-5_8-13_3ch_0.6852_55.1282.pth"

# MI
net.load_state_dict(torch.load(online_path))

<All keys matched successfully>

In [4]:
parms = net.state_dict()
parms.keys()

odict_keys(['layer1.0.weight', 'layer1.0.bias', 'layer1.1.weight', 'layer1.1.bias', 'layer1.1.running_mean', 'layer1.1.running_var', 'layer1.1.num_batches_tracked', 'layer2.0.weight', 'layer2.0.bias', 'layer2.1.weight', 'layer2.1.bias', 'layer2.1.running_mean', 'layer2.1.running_var', 'layer2.1.num_batches_tracked', 'layer3.0.weight', 'layer3.0.bias', 'layer3.1.weight', 'layer3.1.bias', 'layer3.1.running_mean', 'layer3.1.running_var', 'layer3.1.num_batches_tracked', 'fc.weight', 'fc.bias'])

In [5]:
for name,param in net.named_parameters():
    if param.requires_grad and 'layer1' in name:
        param.requires_grad = False
    if param.requires_grad and 'layer2' in name:
        param.requires_grad = False

In [6]:
# home directory + datasets folder
#path = "./dataset/finetune_EEG/"
path = RECORED_PATH
#subject to run
#runs = [3,5,7,9]
runs = [14,15,16,17,18]
subjects = [18]
#recorded eeg class
eeg = EEG(path, subjects, runs)
raw=eeg.data_to_raw()

print("Raw done")
# apply filter 
X,y = eeg.raw_preprocess(raw)
X = eeg.apply_baseline(X)
# iir_param = dict(order=5, ftype='butter', output='sos')
# raw=raw.notch_filter([50])
# #raw=raw.filter(8,14,method = 'iir',iir_params=iir_param,phase='zero')
# raw = raw.filter(8,14, method='fir', verbose=20)
print("Filter done")
# # apply filter 
# iir_param = dict(order=5, ftype='butter', output='sos')
# raw=raw.notch_filter([50])
# #raw=raw.filter(8,14,method = 'iir',iir_params=iir_param,phase='zero')
# raw = raw.filter(8,14, method='fir', verbose=20)
# print("Filter done")

# epochs=eeg.epochs(raw,tmin=-3,tmax=5,baseline=(-3,0))
# #X = X[:, :,np.newaxis,:]
# X, y = eeg.get_X_y(epochs)

/Users/Pongk/Desktop/Work/mi-project/EEG-python/dataset/recorded_EEG
Raw done
Filter done


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y)

print('train size',X_train.shape, y_train.shape)
print('Test size',X_test.shape, y_test.shape)

train size (70, 3, 1750) (70,)
Test size (30, 3, 1750) (30,)


In [8]:
batch_size = 1750

train_loader = create_dataloader(X_train, y_train, batch_size=batch_size)
test_loader = create_dataloader(X_test, y_test, batch_size=batch_size)

num_step =math.ceil(len(train_loader.dataset) / batch_size)

In [9]:
import wandb
wandb.login()


wand = wandb.init(
        
      # Set the project where this run will be logged
      project="Motor-Imagery", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"CNN_S18_Finetune_conv2_online_2ch", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.0000001,
      "architecture": "CNN",
      "dataset": "Finetune",
      "epochs": 300000,
      "weightname":"S18_Finetune_conv2_online_3ch",
      "num_step_per_epoch" : num_step, 
        
      }
    )

config = wand.config
print(config.num_step_per_epoch)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pongkorn-set. Use `wandb login --relogin` to force relogin


1


In [10]:
from utils import train
config = wand.config
optimizer = optim.Adam(net.parameters(), lr=config.learning_rate)
criterion = nn.CrossEntropyLoss()
device = 'cuda'

train_loss,valid_loss,train_accuracy,valid_accuracy =train(
    model = net,
    loader_train = train_loader,
    loader_test = test_loader,
    vail_loader = None,
    optimizer = optimizer  ,
    criterion = criterion ,
    device = 'cuda',
    wand = wand
)

wandb.alert(
            title='Finish',
            text=f'Finishing training',
        )

Epoch 1/300000, Tr Loss: 0.8523, Tr Acc: 42.8571, Val Loss: 0.6989, Val Acc: 46.6667
Epoch 101/300000, Tr Loss: 0.8048, Tr Acc: 47.1429, Val Loss: 0.7053, Val Acc: 46.6667
Epoch 201/300000, Tr Loss: 0.8122, Tr Acc: 47.1429, Val Loss: 0.7051, Val Acc: 46.6667
Epoch 301/300000, Tr Loss: 0.8259, Tr Acc: 41.4286, Val Loss: 0.7052, Val Acc: 46.6667
Epoch 401/300000, Tr Loss: 0.7668, Tr Acc: 51.4286, Val Loss: 0.7051, Val Acc: 46.6667
Epoch 501/300000, Tr Loss: 0.7841, Tr Acc: 47.1429, Val Loss: 0.7048, Val Acc: 46.6667
Epoch 601/300000, Tr Loss: 0.7561, Tr Acc: 47.1429, Val Loss: 0.7050, Val Acc: 46.6667
Epoch 701/300000, Tr Loss: 0.7673, Tr Acc: 50.0000, Val Loss: 0.7049, Val Acc: 46.6667
Epoch 801/300000, Tr Loss: 0.7899, Tr Acc: 54.2857, Val Loss: 0.7050, Val Acc: 46.6667
Epoch 901/300000, Tr Loss: 0.7727, Tr Acc: 48.5714, Val Loss: 0.7051, Val Acc: 46.6667
Epoch 1001/300000, Tr Loss: 0.7578, Tr Acc: 50.0000, Val Loss: 0.7051, Val Acc: 46.6667
Epoch 1101/300000, Tr Loss: 0.7417, Tr Acc: 